In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import seaborn as sns



In [ ]:
#setting our columns to display full values
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import numpy as np
import pandas as pd
from helper import TelecomHelper
from clean_telecom_df import CleanTelecomData


In [ ]:
CSV_PATH = "../data/data.csv"

In [ ]:
Helper = TelecomHelper()

In [ ]:
df = Helper.read_csv(CSV_PATH, missing_values=["n/a", "na", "undefined"])

In [ ]:
pd.set_option('max_column', None)

In [ ]:
df.head()

In [ ]:
def convert_labels(df):
        df.columns = [column.replace(' ', '_').lower() for column in df.columns]
        return df

In [ ]:
df = convert_labels(df)

In [ ]:
class OverViewAnalysis:
    
    def __init__(self, df):
        
        self.df = df
    
    
    def read_head(self, top=5):
        return self.df.head(top)
    
    # returning the number of rows columns and column information
    def get_info(self):
        row_count, col_count = self.df.shape
    
        print(f"Number of rows: {row_count}")
        print(f"Number of columns: {col_count}")
        print("================================")

        return (row_count, col_count), self.df.info()
    
    # gets number of distnict values in a given coumn
    def get_count(self, column_name):
        return self.df[column_name].value_counts()
    
    def get_null_count(self, column_name):
        print("Null values count")
        print(self.df.isnull().sum())
        return self.df.isnull().sum()
    
    def get_percent_missing(self):
        Helper = TelecomHelper()
        
        percent_missing = Helper.percent_missing(self.df)
        
        null_percent_df = pd.DataFrame(columns = ['column', 'null_percent'])
        columns = self.df.columns.values.tolist()
        
        null_percent_df['column'] = columns
        null_percent_df['null_percent'] = null_percent_df['column'].map(lambda x: Helper.percent_missing_for_col(self.df, x))
        
        
        return null_percent_df.sort_values(by=['null_percent'], ascending = False), percent_missing
    
    
    def top_handset_type(self, top=5):
        
        return self.df['handset_type'].value_counts().head(top)
    
    def top_manufacturer(self, top=5):
        
        return self.df['handset_manufacturer'].value_counts().head(top)
    
    def top_handset_by_manufacturer(self, manufacturer, top=5):
        
        return df.groupby('handset_manufacturer')['handset_type'].value_counts()[manufacturer].head(top)
    
        
    
        
    
 
 

In [ ]:
overViewAnal = OverViewAnalysis(df)

In [ ]:
overViewAnal.read_head()

In [ ]:
overViewAnal.get_info()

In [ ]:
df.isnull().sum()

In [ ]:
null_percent_df, overall_missing = overViewAnal.get_percent_missing()
print(f"The overall missing perceentage is: {overall_missing}%")
print("=============================================")
null_percent_df


In [ ]:
cleanTelecomData = CleanTelecomData(df)

In [ ]:

cleaned_df = cleanTelecomData.drop_columns_with_null_values(df)

cleaned_df = cleanTelecomData.convert_to_datetime(cleaned_df)

cleaned_df = cleanTelecomData.drop_duplicate(cleaned_df)

cleaned_df = cleanTelecomData.drop_rows_with_null_values(cleaned_df)

cleaned_df = cleanTelecomData.handle_missing_qantitative_data_with_mean(cleaned_df)

cleaned_df = cleanTelecomData.handle_missing_categorical_data_with_mean(cleaned_df, "ffill")



overViewAnal_2 = OverViewAnalysis(cleaned_df)


In [ ]:
null_percent_df, overall_missing = overViewAnal_2.get_percent_missing()
print(f"The overall missing perceentage is: {overall_missing}%")
print("=============================================")

In [ ]:
print(f"percent of null value rows in last_location_name column: {Helper.percent_missing_for_col(cleaned_df, 'last_location_name')}%"
     )
print(f"percent of null value rows in handset_manufacturer: {Helper.percent_missing_for_col(cleaned_df, 'handset_manufacturer')}%")
print(f"percent of null value rows in handset_type: {Helper.percent_missing_for_col(cleaned_df, 'handset_type')}%")
print(f"percent of null value rows in start: {Helper.percent_missing_for_col(cleaned_df, 'start')}%")
print(f"percent of null value rows in end: {Helper.percent_missing_for_col(cleaned_df, 'end')}%")




In [ ]:
overViewAnal.top_handset_type(top=10)

In [ ]:
overViewAnal.top_manufacturer(top=3)

In [ ]:
overViewAnal.top_handset_by_manufacturer('Apple', 5)

In [ ]:
overViewAnal.top_handset_by_manufacturer('Samsung', 5)

In [ ]:
overViewAnal.top_handset_by_manufacturer('Huawei', 5)

In [ ]:
cleaned_df['bearer_id'].isnull().any()
print(f"percent missing for bearer_id is:{Helper.percent_missing_for_col(cleaned_df, 'bearer_id')}")

In [ ]:
print(f"percent missing for msisdn/number is:{Helper.percent_missing_for_col(cleaned_df, 'msisdn/number')}")

In [ ]:
cleaned_df = cleaned_df.dropna(subset=['msisdn/number'])
print(f"percent missing for msisdn/number is:{Helper.percent_missing_for_col(cleaned_df, 'msisdn/number')}")

In [ ]:
sessions = cleaned_df.groupby('msisdn/number').agg({'bearer_id': 'count'})
sessions = sessions.rename(columns= {'bearer_id': 'xDR_sessions'})
sessions.sort_values(by=['xDR_sessions'], ascending = False).head(10)

In [ ]:
print(f"percent missing for dur._(ms) is:{Helper.percent_missing_for_col(cleaned_df, 'dur._(ms)')}")

In [ ]:
durations = cleaned_df.groupby('msisdn/number').agg({'dur._(ms)': 'sum'})
#here is the duration for first five rows in our dataset
durations=durations.rename(columns= {'dur._(ms)': 'total_duration(ms)'})
durations.sort_values(by=['total_duration(ms)'], ascending = False).head(10)

In [ ]:
total_data_df = cleaned_df[['msisdn/number','total_ul_(bytes)','total_dl_(bytes)']].copy()
total_data_df = total_data_df.groupby('msisdn/number').sum()
total_data_df['total_data(bytes)'] = total_data_df['total_ul_(bytes)'] + total_data_df['total_dl_(bytes)']
total_data_df.sort_values(by=['total_data(bytes)'], ascending = False).head(10)

In [ ]:
cleaned_df["social_media"] = cleaned_df["social_media_dl_(bytes)"] + cleaned_df['social_media_ul_(bytes)']
cleaned_df["google"] = cleaned_df["google_dl_(bytes)"] + cleaned_df["google_ul_(bytes)"]
cleaned_df['email'] = cleaned_df["email_dl_(bytes)"] + cleaned_df["email_ul_(bytes)"]
cleaned_df['youtube'] = cleaned_df["youtube_dl_(bytes)"] + cleaned_df["youtube_ul_(bytes)"]
cleaned_df['netflix'] = cleaned_df["netflix_dl_(bytes)"] + cleaned_df["netflix_ul_(bytes)"]
cleaned_df["gaming"] = cleaned_df["gaming_dl_(bytes)"] + cleaned_df["gaming_ul_(bytes)"]
cleaned_df['other']= cleaned_df["other_dl_(bytes)"]+ cleaned_df["other_ul_(bytes)"]
cleaned_df['total_data'] = cleaned_df['total_dl_(bytes)'] + cleaned_df['total_ul_(bytes)']

In [ ]:
app_total_data = cleaned_df[['msisdn/number','social_media','google',
                             'email', 'youtube','netflix', 'gaming','other']].copy()

In [ ]:
app_total_data.groupby('msisdn/number').sum().sample(10)

In [ ]:
SAVE_PATH = "../data/cleaned_data.csv"

In [ ]:
cleaned_df =Helper.save_csv(cleaned_df, SAVE_PATH)

In [ ]:
Helper.percent_missing(cleaned_df)

In [ ]:
cleaned_df.info()